# Project 3

## (Task 0: Loading and preparing sample data)

## Task 1: Preprocess data to format according spec

In [42]:
import numpy as np
import matplotlib.pyplot as plt

# example_path = 'enrollment/001-g-01.txt'

# Accept as input a number of one user as string, e.g '001'
# Output format as [[[userId, [[x1, y1, vx1, vy1, pressure1], [x2, y2, vx2, vy2, pressure2], [...], ...]]]]
# -> userId = '001-01' (user 001 genuine signature 01) -> one user has 5 genuine signature
# To get the array with all the signature data for dtw -> output[i][0][1]
def preprocess_enrollment(usernumber):
    data = []
    
    for i in range(5):
        path = 'enrollment/' + str(usernumber) + '-g-0' + str(i+1) + '.txt'
        signature = []
        with open(path, "r") as lines:
            for line in lines:
                signature.append(line.split())
        data.append([str(usernumber) + '-g-0' + str(i+1), signature])
        
    processed_data = preprocess_data(data)
    
    return processed_data


# Accept as input a number of one user as string, e.g '001'
# Output format as [[[userId, [[x1, y1, vx1, vy1, pressure1], [x2, y2, vx2, vy2, pressure2], [...], ...]]]]
# -> userId = '001-01' (user 001 signature 01 for verification) -> one user has 45 signature
# To get the array with all the signature data for dtw -> output[i][0][1]
def preprocess_verification(usernumber):
    data = []
    
    for i in range(45):
        if i+1 < 10:
            userId = str(usernumber) + '-0' + str(i+1)
        else:
            userId = str(usernumber) + '-' + str(i+1)
        signature = []    
        path = 'verification/' + userId + '.txt'
        with open(path, "r") as lines:
            for line in lines:
                signature.append(line.split())
        data.append([userId, signature])
        
    processed_data = preprocess_data(data)
    
    return processed_data


def preprocess_data(data):
    data_for_dtw = []
    
    for user in data:
        vx = 0
        vy = 0
        x = 0
        y = 0 
        time = 0
        
        input_data = []
        for signature_data in user[1]:
            new_time = float(signature_data[0])
            new_x = float(signature_data[1])
            new_y = float(signature_data[2])
            pressure = float(signature_data[3])
            penup = float(signature_data[4])
            if penup == 1:
                vx = 0
                vy = 0
                input_data.append([new_x, new_y, vx, vy, pressure])
                x = new_x
                y = new_y
                time = new_time
            else:
                vx = float((new_x - x)/(new_time - time))
                vy = float((new_y - y)/(new_time - time))
                input_data.append([new_x, new_y, vx, vy, pressure])
                x = new_x
                y = new_y
                time = new_time 
                
        data_for_dtw.append([user, input_data])
            
    return data_for_dtw

In [33]:
test = preprocess_enrollment('001')

In [47]:
print(test[1][0][1])

[['0.0000', '16.1700', '63.2500', '170.000000', '1.000000', '115.000000', '58.000000'], ['0.0100', '17.7000', '64.8500', '257.000000', '0.000000', '115.000000', '58.000000'], ['0.0200', '17.8500', '65.2300', '344.000000', '0.000000', '116.000000', '58.000000'], ['0.0300', '17.5900', '65.3400', '444.000000', '0.000000', '116.000000', '58.000000'], ['0.0400', '17.4900', '65.2600', '488.000000', '0.000000', '117.000000', '59.000000'], ['0.0500', '17.1900', '65.1300', '526.000000', '0.000000', '119.000000', '60.000000'], ['0.0600', '16.7600', '64.2100', '543.000000', '0.000000', '119.000000', '60.000000'], ['0.0700', '15.5300', '62.7700', '567.000000', '0.000000', '119.000000', '60.000000'], ['0.0800', '13.9800', '60.4900', '560.000000', '0.000000', '121.000000', '59.000000'], ['0.0900', '12.2200', '57.6800', '515.000000', '0.000000', '119.000000', '60.000000'], ['0.1000', '10.7700', '54.5300', '466.000000', '0.000000', '118.000000', '59.000000'], ['0.1100', '9.4900', '51.9100', '468.00000

In [43]:
test2 = preprocess_verification('001')

In [49]:
print(test2)

[[['001-01', [['0.0000', '18.0100', '15.9200', '119.000000', '1.000000', '123.000000', '61.000000'], ['0.0100', '19.6200', '18.2500', '170.000000', '0.000000', '125.000000', '60.000000'], ['0.0200', '20.1000', '18.9000', '265.000000', '0.000000', '125.000000', '60.000000'], ['0.0300', '20.4100', '19.2600', '382.000000', '0.000000', '126.000000', '60.000000'], ['0.0400', '20.5100', '19.4200', '448.000000', '0.000000', '128.000000', '59.000000'], ['0.0500', '20.5100', '19.3500', '491.000000', '0.000000', '129.000000', '59.000000'], ['0.0600', '20.3600', '19.2300', '546.000000', '0.000000', '129.000000', '59.000000'], ['0.0700', '19.8000', '18.2400', '581.000000', '0.000000', '129.000000', '59.000000'], ['0.0800', '18.7700', '16.7000', '589.000000', '0.000000', '131.000000', '58.000000'], ['0.0900', '17.4300', '14.5100', '592.000000', '0.000000', '131.000000', '58.000000'], ['0.1000', '15.8300', '12.1400', '595.000000', '0.000000', '129.000000', '59.000000'], ['0.1100', '14.1500', '9.6600

## Task 2: Dynamic Time Warping for Signatures

## Task 3: Apply and train

## Task 4: Combine everything and run